In [1]:
%load_ext autoreload

In [2]:
from certified_iris_generator import CertifiedIrisRegionGenerator
import sys
import os
import time
import numpy as np
from functools import partial
import itertools
import mcubes
import visualizations_utils as viz_utils
import iris_utils #TODO remove
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from sandbox import rrtiris

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.all import InverseKinematics, RevoluteJoint, RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptionsRationalSpace, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
import pydrake.symbolic as sym
from pydrake.all import MathematicalProgram, RigidTransform, RollPitchYaw
import meshcat

# Build Plant


In [4]:
q0 = [0.0, 0.0, 0.0]
q_low  = [-1.7, -2., -1.7]
q_high = [ 1.7,  2.,  1.7]


In [5]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
oneDOF_iiwa_asset = FindResourceOrThrow("drake/sos_iris_certifier/assets/oneDOF_iiwa7_with_box_collision.sdf")
twoDOF_iiwa_asset = FindResourceOrThrow("drake/sos_iris_certifier/assets/twoDOF_iiwa7_with_box_collision.sdf")

In [6]:
box_asset = FindResourceOrThrow("drake/sos_iris_certifier/assets/box_small.urdf")

models = []
models.append(parser.AddModelFromFile(box_asset))
models.append(parser.AddModelFromFile(twoDOF_iiwa_asset))
models.append(parser.AddModelFromFile(oneDOF_iiwa_asset))



locs = [[0.,0.,0.],[0.,.5,0.],[0.,-.5,0.]]
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base", models[0]), RigidTransform(locs[0]))
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("iiwa_twoDOF_link_0", models[1]), RigidTransform(RollPitchYaw([0,0, -np.pi/2]).ToRotationMatrix(), locs[1]))
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("iiwa_oneDOF_link_0", models[2]), RigidTransform(RollPitchYaw([0,0, -np.pi/2]).ToRotationMatrix(), locs[2]))


plant.Finalize()

idx = 0
for model in models:
    for joint_index in plant.GetJointIndices(model):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[idx])
            joint.set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([q_high[idx]]))
            idx += 1
        
            
#q_low = plant.GetPositionLowerLimits().tolist()
#q_high = plant.GetPositionUpperLimits().tolist()

Ratfk = RationalForwardKinematics(plant)


In [7]:
do_viz = True
#visualizer = IrisPlantVisualizer(plant, builder, scene_graph, viz_role=Role.kProximity)
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, viz_role=Role.kProximity)
diagram = visualizer.diagram
visualizer.visualize_collision_constraint(N = 30)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.


In [8]:
sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))
sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    #print(q, end="\r")
    visualizer.showres(q)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)

visualizer.jupyter_cell()

FloatSlider(value=0.0, description='q0', max=1.7, min=-1.7)

FloatSlider(value=0.0, description='q1', max=2.0, min=-2.0)

FloatSlider(value=0.0, description='q2', max=1.7, min=-1.7)

In [9]:
#filter fused joints self collisions 
digaram = visualizer.diagram
context = visualizer.diagram_context
plant_context = visualizer.plant_context
sg_context = scene_graph.GetMyContextFromRoot(context)
inspector = scene_graph.model_inspector()

pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()
print(len(inspector.GetCollisionCandidates()), "->", len(pairs))

gids = [gid for gid in inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)]
get_name_of_gid = lambda gid : inspector.GetName(gid)
gids.sort(key=get_name_of_gid)
iiwa_oneDOF_gids = [gid for gid in gids if "iiwa7_oneDOF::" in get_name_of_gid(gid)]
iiwa_twoDOF_gids = [gid for gid in gids if "iiwa7_twoDOF::" in get_name_of_gid(gid)]

oneDOF_fused_col_geom = iiwa_oneDOF_gids[2:]
iiwa_oneDOF_fused_set = GeometrySet(oneDOF_fused_col_geom)
twoDOF_fused_col_geom = iiwa_twoDOF_gids[4:]
iiwa_twoDOF_fused_set = GeometrySet(twoDOF_fused_col_geom)
# print([get_name_of_gid(gid) for gid in oneDOF_fused_col_geom])
# print([get_name_of_gid(gid) for gid in twoDOF_fused_col_geom])
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(iiwa_oneDOF_fused_set))
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(iiwa_twoDOF_fused_set))
pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()
print(len(inspector.GetCollisionCandidates()), "->", len(pairs))

150 -> 150
137 -> 150


In [10]:
gid_endeff_oneDof = iiwa_oneDOF_gids[-1]
gid_endeff_twoDof = iiwa_twoDOF_gids[-1]


# Setup IRIS Options and Generate Regions

In [20]:
seed_points_q = np.array([[1.0, -1.2, -0.3],[0.3,-0.9, -0.3]])
seed_points = np.zeros_like(seed_points_q)
for idx in range(seed_points_q.shape[0]):
    seed_points[idx, :] = Ratfk.ComputeTValue(seed_points_q[idx,:], np.zeros((3,)))
generator = CertifiedIrisRegionGenerator(diagram, plant, scene_graph, iris_max_faces = 500)
generator.iris_in_rational_space(seed_points, termination_threshold=1e-4)
visualizer.plot_regions(generator.regions, ellipses=None, region_suffix='py', opacity=0.8)
visualizer.plot_seedpoints(seed_points)
generator.initalize_certifier(plane_order=1)

snopt_example=[ 0.37381737 -0.56336652 -0.24240893], growth = 52667.469298131764
snopt_example=[ 0.45644167 -0.67889049 -0.55138103], growth = 168299.19683477422
snopt_example=[ 0.45390959 -0.54999184 -0.37604167], growth = 77114.23364629946
snopt_example=[ 0.45088097 -0.55613842 -0.37845126], growth = 77161.43876081635
snopt_example=[ 0.45103995 -0.55581487 -0.3783236 ], growth = 77156.02942476119
snopt_example=[ 0.40376233 -0.55997561 -0.28304097], growth = 53132.82922674436
snopt_example=[ 0.52009081 -0.4999248  -0.30073701], growth = 57001.81186332379
snopt_example=[ 0.75489324 -0.7556322  -0.15113522], growth = 48621.69306437997
snopt_example=[ 0.52909786 -0.83565877 -0.5686664 ], growth = 197587.18925424176
snopt_example=[ 0.54630249 -0.68413681  0.39746   ], growth = 300956.71132488793
snopt_example=[ 0.54630249 -0.68413681  0.22974538], growth = 145070.02677341423
snopt_example=[ 0.72652942 -0.75382598 -0.15113521], growth = 37338.324666227214
snopt_example=[ 0.20196462 -1.0125

{<pydrake.geometry.optimization.HPolyhedron at 0x7f2feb2ea0f0>: <certified_iris_generator.RegionCertificationProblem at 0x7f2feae567c0>,
 <pydrake.geometry.optimization.HPolyhedron at 0x7f2feae9bf30>: <certified_iris_generator.RegionCertificationProblem at 0x7f2fea8726a0>}

In [21]:
generator.certify_and_adjust_regions_by_linesearch()

Starting Region 1/2
min_eps = [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
cur_eps = [0.29601511 1.12077227 0.64473397 0.8423176  0.43663546 0.49359875
 0.11474697 0.20512143 0.13884725 0.13888974 0.13888487 0.1152528
 0.11937526 0.11025164 0.2222539  0.27429761 0.1904403  0.09661564
 0.29237265 0.10427334 0.1043285  0.10432361 0.24228607 0.16726003
 0.12440679 0.12440195 0.09140995]
max_eps = [0.59203022 2.24154453 1.28946793 1.6846352  0.87327092 0.9871975
 0.22949394 0.41024285 0.2776945  0.27777948 0.27776974 0.23050559
 0.23875052 0.22050327 0.44450781 0.54859522 0.38088059 0.19323127
 0.5847453  0.20854668 0.20865701 0.20864721 0.48457214 0.33452006
 0.24881357 0.24880389 0.18281991]
Biggest difference: 2.2415445329965946
Smallest difference: 0.18281990625371392
Fixed eps is success

min_eps = [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
cur_eps = [0.14800756 0.56038613 0.32236698 0.4211588  0.21831773 

Fixed eps is not success

min_eps = [0.00173446 0.00656702 0.00377774 0.00493545 0.00255841 0.00289218
 0.00067235 0.00120188 0.00081356 0.00081381 0.00081378 0.00067531
 0.00069946 0.00064601 0.00130227 0.00160721 0.00111586 0.00056611
 0.00171312 0.00061098 0.0006113  0.00061127 0.00141964 0.00098004
 0.00072895 0.00072892 0.00053561]
cur_eps = [0.00202354 0.00766153 0.00440736 0.00575803 0.00298481 0.00337421
 0.0007844  0.0014022  0.00094915 0.00094944 0.00094941 0.00078786
 0.00081604 0.00075367 0.00151931 0.00187508 0.00130184 0.00066046
 0.00199864 0.00071281 0.00071318 0.00071315 0.00165625 0.00114338
 0.00085044 0.0008504  0.00062487]
max_eps = [0.00231262 0.00875603 0.00503698 0.00658061 0.00341121 0.00385624
 0.00089646 0.00160251 0.00108474 0.00108508 0.00108504 0.00090041
 0.00093262 0.00086134 0.00173636 0.00214295 0.00148781 0.00075481
 0.00228416 0.00081464 0.00081507 0.00081503 0.00189286 0.00130672
 0.00097193 0.00097189 0.00071414]
Biggest difference: 0.0021890083330

Fixed eps is success

min_eps = [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
cur_eps = [0.01419981 0.01141473 0.01253067 0.01117326 0.01592144 0.00887513
 0.00170795 0.00170817 0.01269843 0.01068849 0.00243323 0.01191536
 0.00979389 0.00979374 0.00857178 0.00595127 0.00927213 0.00149029
 0.0102358  0.01395938 0.0069969 ]
max_eps = [0.02839962 0.02282946 0.02506133 0.02234651 0.03184287 0.01775025
 0.00341589 0.00341635 0.02539685 0.02137698 0.00486646 0.02383071
 0.01958779 0.01958748 0.01714356 0.01190253 0.01854426 0.00298059
 0.02047161 0.02791877 0.0139938 ]
Biggest difference: 0.03184287028165664
Smallest difference: 0.0029805852138161614
Fixed eps is success

min_eps = [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
cur_eps = [0.00709991 0.00570736 0.00626533 0.00558663 0.00796072 0.00443756
 0.00085397 0.00085409 0.00634921 0.00534425 0.00121662 0.00595768
 0.00489695 0.00489687 0.00428589 0.00297563 0.00463607 0.00074515
 0.0051179  0.006979

In [22]:
regions = generator.linesearch_regions
visualizer.region_to_collision_pair_to_plane_dictionary = generator.linesearch_regions_to_certificates_by_collision_pair_map
print(gid_endeff_twoDof, gid_endeff_oneDof)

<GeometryId value=92> <GeometryId value=140>


In [23]:
sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))
sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    visualizer.show_res_with_planes_gid(q, gid_endeff_twoDof)
    #visualizer.show_res_with_planes(q)
    
idx = 0
for slider in sliders:
    a = slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)

visualizer.jupyter_cell()

FloatSlider(value=0.0, description='q0', max=1.7, min=-1.7)

FloatSlider(value=0.0, description='q1', max=2.0, min=-2.0)

FloatSlider(value=0.0, description='q2', max=1.7, min=-1.7)

0*1 + 0*t[2] + 0*t[1] + 0*t[0]
{Variable('t[0]', Continuous): 0.05004170837553879, Variable('t[1]', Continuous): 0.0, Variable('t[2]', Continuous): 0.0}

0*1 + 0*t[2] + 0*t[1] + 0*t[0]
{Variable('t[0]', Continuous): 0.05004170837553879, Variable('t[1]', Continuous): 0.0, Variable('t[2]', Continuous): 0.0}

0*1 + 0*t[2] + 0*t[1] + 0*t[0]
{Variable('t[0]', Continuous): 0.05004170837553879, Variable('t[1]', Continuous): 0.0, Variable('t[2]', Continuous): 0.0}

0*1 + 0*t[2] + 0*t[1] + 0*t[0]
{Variable('t[0]', Continuous): 0.05004170837553879, Variable('t[1]', Continuous): 0.0, Variable('t[2]', Continuous): 0.0}

0*1 + 0*t[2] + 0*t[1] + 0*t[0]
{Variable('t[0]', Continuous): 0.05004170837553879, Variable('t[1]', Continuous): 0.0, Variable('t[2]', Continuous): 0.0}

0*1 + 0*t[2] + 0*t[1] + 0*t[0]
{Variable('t[0]', Continuous): 0.05004170837553879, Variable('t[1]', Continuous): 0.0, Variable('t[2]', Continuous): 0.0}

0*1 + 0*t[2] + 0*t[1] + 0*t[0]
{Variable('t[0]', Continuous): 0.050041708375

/home/peter/.cache/bazel/_bazel_peter/2e631950b59c04e12267dc0dff33d287/execroot/drake/bazel-out/k8-opt/bin/sos_iris_certifier/pinball_iiwas.runfiles/drake/sos_iris_certifier/iris_plant_visualizer.py:217: RuntimeWarning: invalid value encountered in double_scalars
  alpha = (-b - a.T @ p1) / (a.T @ (p2 - p1))


...

0*1 + 0*t[2] + 0*t[1] + 0*t[0]
{Variable('t[0]', Continuous): 0.5463024898437905, Variable('t[1]', Continuous): 0.15113521805829508, Variable('t[2]', Continuous): 0.0}

0*1 + 0*t[2] + 0*t[1] + 0*t[0]
{Variable('t[0]', Continuous): 0.5463024898437905, Variable('t[1]', Continuous): 0.15113521805829508, Variable('t[2]', Continuous): 0.0}

0*1 + 0*t[2] + 0*t[1] + 0*t[0]
{Variable('t[0]', Continuous): 0.5463024898437905, Variable('t[1]', Continuous): 0.15113521805829508, Variable('t[2]', Continuous): 0.0}

0*1 + 0*t[2] + 0*t[1] + 0*t[0]
{Variable('t[0]', Continuous): 0.5463024898437905, Variable('t[1]', Continuous): 0.15113521805829508, Variable('t[2]', Continuous): 0.0}

0*1 + 0*t[2] + 0*t[1] + 0*t[0]
{Variable('t[0]', Continuous): 0.5463024898437905, Variable('t[1]', Continuous): 0.15113521805829508, Variable('t[2]', Continuous): 0.0}

0*1 + 0*t[2] + 0*t[1] + 0*t[0]
{Variable('t[0]', Continuous): 0.5463024898437905, Variable('t[1]', Continuous): 0.15113521805829508, Variable('t[2]', Conti

0*1 + 0*t[2] + 0*t[1] + 0*t[0]
{Variable('t[0]', Continuous): 0.5463024898437905, Variable('t[1]', Continuous): 0.10033467208545055, Variable('t[2]', Continuous): 0.0}

0*1 + 0*t[2] + 0*t[1] + 0*t[0]
{Variable('t[0]', Continuous): 0.5463024898437905, Variable('t[1]', Continuous): 0.10033467208545055, Variable('t[2]', Continuous): 0.0}

0*1 + 0*t[2] + 0*t[1] + 0*t[0]
{Variable('t[0]', Continuous): 0.5463024898437905, Variable('t[1]', Continuous): 0.10033467208545055, Variable('t[2]', Continuous): 0.0}

0*1 + 0*t[2] + 0*t[1] + 0*t[0]
{Variable('t[0]', Continuous): 0.5463024898437905, Variable('t[1]', Continuous): 0.10033467208545055, Variable('t[2]', Continuous): 0.0}

0*1 + 0*t[2] + 0*t[1] + 0*t[0]
{Variable('t[0]', Continuous): 0.5463024898437905, Variable('t[1]', Continuous): 0.10033467208545055, Variable('t[2]', Continuous): 0.0}

0*1 + 0*t[2] + 0*t[1] + 0*t[0]
{Variable('t[0]', Continuous): 0.5463024898437905, Variable('t[1]', Continuous): 0.10033467208545055, Variable('t[2]', Conti

0*1 + 0*t[2] + 0*t[1] + 0*t[0]
{Variable('t[0]', Continuous): 0.5463024898437905, Variable('t[1]', Continuous): 0.05004170837553879, Variable('t[2]', Continuous): 0.0}

0*1 + 0*t[2] + 0*t[1] + 0*t[0]
{Variable('t[0]', Continuous): 0.5463024898437905, Variable('t[1]', Continuous): 0.05004170837553879, Variable('t[2]', Continuous): 0.0}

0*1 + 0*t[2] + 0*t[1] + 0*t[0]
{Variable('t[0]', Continuous): 0.5463024898437905, Variable('t[1]', Continuous): 0.05004170837553879, Variable('t[2]', Continuous): 0.0}

0*1 + 0*t[2] + 0*t[1] + 0*t[0]
{Variable('t[0]', Continuous): 0.5463024898437905, Variable('t[1]', Continuous): 0.05004170837553879, Variable('t[2]', Continuous): 0.0}

0*1 + 0*t[2] + 0*t[1] + 0*t[0]
{Variable('t[0]', Continuous): 0.5463024898437905, Variable('t[1]', Continuous): 0.05004170837553879, Variable('t[2]', Continuous): 0.0}

0*1 + 0*t[2] + 0*t[1] + 0*t[0]
{Variable('t[0]', Continuous): 0.5463024898437905, Variable('t[1]', Continuous): 0.05004170837553879, Variable('t[2]', Conti

0*1 + 0*t[2] + 0*t[1] + 0*t[0]
{Variable('t[0]', Continuous): 0.5463024898437905, Variable('t[1]', Continuous): 0.0, Variable('t[2]', Continuous): 0.0}

0*1 + 0*t[2] + 0*t[1] + 0*t[0]
{Variable('t[0]', Continuous): 0.5463024898437905, Variable('t[1]', Continuous): 0.0, Variable('t[2]', Continuous): 0.0}

0*1 + 0*t[2] + 0*t[1] + 0*t[0]
{Variable('t[0]', Continuous): 0.5463024898437905, Variable('t[1]', Continuous): 0.0, Variable('t[2]', Continuous): 0.0}

0*1 + 0*t[2] + 0*t[1] + 0*t[0]
{Variable('t[0]', Continuous): 0.5463024898437905, Variable('t[1]', Continuous): 0.0, Variable('t[2]', Continuous): 0.0}

0*1 + 0*t[2] + 0*t[1] + 0*t[0]
{Variable('t[0]', Continuous): 0.5463024898437905, Variable('t[1]', Continuous): 0.0, Variable('t[2]', Continuous): 0.0}

0*1 + 0*t[2] + 0*t[1] + 0*t[0]
{Variable('t[0]', Continuous): 0.5463024898437905, Variable('t[1]', Continuous): 0.0, Variable('t[2]', Continuous): 0.0}

0*1 + 0*t[2] + 0*t[1] + 0*t[0]
{Variable('t[0]', Continuous): 0.5463024898437905, 

0*1 + 0*t[2] + 0*t[1] + 0*t[0]
{Variable('t[0]', Continuous): 0.5463024898437905, Variable('t[1]', Continuous): -0.10033467208545055, Variable('t[2]', Continuous): 0.0}

0*1 + 0*t[2] + 0*t[1] + 0*t[0]
{Variable('t[0]', Continuous): 0.5463024898437905, Variable('t[1]', Continuous): -0.10033467208545055, Variable('t[2]', Continuous): 0.0}

0*1 + 0*t[2] + 0*t[1] + 0*t[0]
{Variable('t[0]', Continuous): 0.5463024898437905, Variable('t[1]', Continuous): -0.10033467208545055, Variable('t[2]', Continuous): 0.0}

0*1 + 0*t[2] + 0*t[1] + 0*t[0]
{Variable('t[0]', Continuous): 0.5463024898437905, Variable('t[1]', Continuous): -0.10033467208545055, Variable('t[2]', Continuous): 0.0}

0*1 + 0*t[2] + 0*t[1] + 0*t[0]
{Variable('t[0]', Continuous): 0.5463024898437905, Variable('t[1]', Continuous): -0.10033467208545055, Variable('t[2]', Continuous): 0.0}

0*1 + 0*t[2] + 0*t[1] + 0*t[0]
{Variable('t[0]', Continuous): 0.5463024898437905, Variable('t[1]', Continuous): -0.10033467208545055, Variable('t[2]',

In [15]:
visualizer.show_res_with_planes

<bound method IrisPlantVisualizer.show_res_with_planes of <iris_plant_visualizer.IrisPlantVisualizer object at 0x7fd12fecc3a0>>

In [16]:
# seed_points_q = np.array([[0.0, 0, 0], # zero config
#                         [0.0, -1.3, -1.3],  # start
#                         [0.9, -1.5, -0.8]     # goal
#                          ])
seed_points_q = np.array([[0.0, 0, 0], # zero config
                        [0.8, 1.3, -0.8],  # START: blue low green up
                        [0.1, 0.9, -1.2],     # GOAL: green low other up
                        [0.2, 1.6, -0.6],
                        [0.5, -1.9, -0.9],
                        [1.1, -1.0, -0.9]
                          #
#
                         ])[:,(0,1,2)]   # passing


seed_points = np.array([Ratfk.ComputeTValue(seed_points_q[idx], np.zeros((3,)))\
                        for idx in range(seed_points_q.shape[0])])
if do_viz:
    visualizer.plot_seedpoints(seed_points)

start = seed_points[1,:]
goal = seed_points[2,:]

#compute limits in t-space
limits_t = []
for q in [q_low, q_high]:
    limits_t.append(Ratfk.ComputeTValue(np.array(q), np.zeros((3,)) ))
    
starting_poly = HPolyhedron.MakeBox(limits_t[0], limits_t[1])
context = diagram.CreateDefaultContext()

In [17]:
regions = []

iris_options = IrisOptionsRationalSpace()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 20
iris_options.configuration_space_margin = 1e-6
iris_options.max_faces_per_collision_pair = 60
iris_options.termination_threshold = -1
iris_options.q_star = np.zeros(3)
iris_options.relative_termination_threshold = 0.05
iris_options.enable_ibex = False
#deprecated
iris_options.certify_region_with_sos_during_generation = False
iris_options.certify_region_with_sos_after_generation = False

for i, s in enumerate(seed_points):
    plant.SetPositions(plant.GetMyMutableContextFromRoot(context), s)
    if False:
        #starting_hpolyhedron = regions[i-1]
        r = IrisInRationalConfigurationSpace (plant, plant.GetMyContextFromRoot(context),
                                              iris_options, starting_hpolyhedron)
    else:
        r = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context), iris_options)
    print(f"{r.A()}t <= {r.b()}")
    regions.append(r)
    print(f'Completed region: {i+1}/{len(seed_points)}')

[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]
 [-1.00000000e+00 -0.00000000e+00 -0.00000000e+00]
 [-0.00000000e+00 -1.00000000e+00 -0.00000000e+00]
 [-0.00000000e+00 -0.00000000e+00 -1.00000000e+00]
 [-5.90019798e-08  3.95004907e-07  1.00000000e+00]
 [-3.53005414e-01  1.76913637e-01  9.18743023e-01]
 [-9.01094976e-01  4.33621775e-01  5.86764971e-08]
 [-9.99939434e-01 -1.10058439e-02 -3.66339133e-07]
 [-9.99939429e-01 -1.10063197e-02  8.39066703e-07]
 [-9.96644322e-01 -2.05374544e-02 -7.92357797e-02]
 [-9.96665012e-01 -2.05073639e-02 -7.89829276e-02]
 [-9.96685707e-01 -2.04769640e-02 -7.87292548e-02]
 [-9.96706402e-01 -2.04464669e-02 -7.84747777e-02]
 [-9.96727096e-01 -2.04158719e-02 -7.82194886e-02]
 [-9.96747790e-01 -2.03851779e-02 -7.79633796e-02]
 [-9.96768483e-01 -2.03543839e-02 -7.77064427e-02]
 [-9.96789175e-01 -2.03234890e-02 -7.74486699e-02]
 [-9.96809867e-01 -2.02924921e-

[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]
 [-1.00000000e+00 -0.00000000e+00 -0.00000000e+00]
 [-0.00000000e+00 -1.00000000e+00 -0.00000000e+00]
 [-0.00000000e+00 -0.00000000e+00 -1.00000000e+00]
 [-9.05103841e-01  4.25190588e-01  8.32515177e-08]
 [-9.99497833e-01 -3.16872541e-02  3.92650350e-06]
 [-9.99497783e-01 -3.16888284e-02  5.19343600e-06]
 [-6.30403728e-01 -2.43131590e-01 -7.37209719e-01]
 [-9.99452459e-01 -3.28583602e-02 -3.88719573e-03]
 [-9.99491110e-01 -3.18909207e-02 -7.00017309e-04]
 [-9.99538572e-01 -2.95400158e-02  7.07315874e-03]
 [-9.99533507e-01 -2.98691259e-02  6.37216691e-03]
 [-9.99534821e-01 -2.99793734e-02  5.60159268e-03]
 [-9.99527462e-01 -3.03715867e-02  4.73481962e-03]
 [-9.99527855e-01 -3.04981057e-02  3.73259532e-03]
 [-9.99512262e-01 -3.11284480e-02  2.50163840e-03]
 [-9.99517027e-01 -3.10644096e-02  8.45873584e-04]
 [-9.99481091e-01 -3.20942282e-

 0.41926801 0.40795221 0.40795738 0.44047322 0.33545563 0.33249854]
Completed region: 4/6
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]
 [-1.00000000e+00 -0.00000000e+00 -0.00000000e+00]
 [-0.00000000e+00 -1.00000000e+00 -0.00000000e+00]
 [-0.00000000e+00 -0.00000000e+00 -1.00000000e+00]
 [ 2.05597487e-01  1.66371813e-01 -9.64391048e-01]
 [ 4.85213250e-02 -2.25921552e-02 -9.98566610e-01]
 [ 3.01047152e-02 -5.79724896e-03 -9.99529939e-01]
 [ 3.01050850e-02 -5.79675642e-03 -9.99529930e-01]
 [ 3.01059091e-02 -5.79662244e-03 -9.99529906e-01]
 [ 3.01067378e-02 -5.79649214e-03 -9.99529882e-01]
 [ 3.01072619e-02 -5.79612135e-03 -9.99529868e-01]
 [ 3.01069927e-02 -5.79512424e-03 -9.99529882e-01]
 [ 3.01052036e-02 -5.79292717e-03 -9.99529949e-01]
 [ 5.62919790e-02  5.58602252e-02 -9.96850464e-01]
 [ 5.62926731e-02  5.58610868e-02 -9.96850377e-01]
 [ 5.62933722e-02  5.58619441e-02 -9.968502

 0.35190391 0.3519014  0.3518989  0.35189639 0.18074348]
Completed region: 5/6
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [-1. -0. -0.]
 [-0. -1. -0.]
 [-0. -0. -1.]]t <= [1.13833271 1.55740772 1.13833271 1.13833271 1.55740772 1.13833271]
Completed region: 6/6


In [18]:
#if do_viz:
#    visualizer.plot_regions(regions, ellipses=None, region_suffix='original', opacity=0.8)

In [19]:
import pydrake.multibody.rational_forward_kinematics as rational_forward_kinematics
from pydrake.multibody.rational_forward_kinematics import FindEpsilonLower
from pydrake.solvers import mathematicalprogram as mp
cspace_free_region = rational_forward_kinematics.CspaceFreeRegion(diagram, plant, scene_graph,
                                   rational_forward_kinematics.SeparatingPlaneOrder.kAffine,
                                   rational_forward_kinematics.CspaceRegionType.kGenericPolytope)

edited_regions = []
filtered_collision_pairs = set()

binary_search_options = rational_forward_kinematics.BinarySearchOption()
binary_search_options.epsilon_max = 5.0
binary_search_options.epsilon_min = -.12
binary_search_options.max_iters = 5
binary_search_options.search_d = True
bilinear_alternation_option = rational_forward_kinematics.BilinearAlternationOption()
bilinear_alternation_option.max_iters = 20
bilinear_alternation_option.lagrangian_backoff_scale = 1e-6
bilinear_alternation_option.polytope_backoff_scale = 1e-6


solver_options = mp.SolverOptions()
solver_options.SetOption(mp.CommonSolverOption.kPrintToConsole, 1)

def iris_handle(seed, domain):
    seed_q = Ratfk.ComputeQValue(seed, np.zeros((3,))) 
    plant.SetPositions(plant.GetMyMutableContextFromRoot(context), seed_q)
    region = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context),
                                              iris_options, domain)
    ellipse = region.MaximumVolumeInscribedEllipsoid()
    C = ellipse.A()
    radii2, R  =  np.linalg.eig(C.T @ C)
    radii = np.sqrt(radii2)
    binary_search_options.epsilon_min = FindEpsilonLower(limits_t[0], limits_t[1], region.A(), region.b())
    print("[IRIS +  CERT]: Eps min", binary_search_options.epsilon_min)
    print("[IRIS +  CERT]: Region found, vol = ", ellipse.Volume(), ", halfaxis: ", radii)
    b_feasible = cspace_free_region.CspacePolytopeBinarySearch(
                 iris_options.q_star, filtered_collision_pairs, region.A(), region.b(),
                 binary_search_options, solver_options)
    
#     C_final, d_final, P_final, q_final = \
#         cspace_free_region.CspacePolytopeBilinearAlternation(
#             iris_options.q_star, filtered_collision_pairs, region.A(), b_feasible,
#            bilinear_alternation_option, solver_options)
#     C_final = np.vstack([C_final, P_joint_limits.A()])
#     d_final = np.concatenate([d_final, P_joint_limits.b()])
#     editted_regions.append(HPolyhedron(C_final, d_final))
    return region, HPolyhedron(region.A(), b_feasible) 

In [20]:
#refine with CPP code:
cpp_editted_regions = []
for i in range(seed_points.shape[0]):
    s = seed_points[i,:]
    region_init = regions[i]
    region_init, region = iris_handle(s, region_init)
    cpp_editted_regions.append(region)

KeyboardInterrupt: 

In [ ]:
if do_viz:
    visualizer.plot_regions(cpp_editted_regions, ellipses=None, region_suffix='cpp_editted_regions')

In [ ]:
#setup refine with python
iris_kwargs = {
    'iris_starting_ellipse_vol': 1e-5,
    'iris_plane_pullback': 1e-4,
    'iris_max_faces': -1
}
iris_generator = CertifiedIrisRegionGenerator(visualizer.diagram, plant, scene_graph, **iris_kwargs)
regions, ellipses = iris_generator.iris_in_rational_space(seed_points)
iris_generator.initalize_certifier(plane_order = 1, strict_pos_tol = 1e-4)

In [ ]:
#cpp

iris_options = IrisOptionsRationalSpace()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 20
iris_options.configuration_space_margin = 0.8e-5
iris_options.max_faces_per_collision_pair = 80
iris_options.termination_threshold = -1
iris_options.q_star = np.zeros(3)
iris_options.relative_termination_threshold = 0.05
iris_options.enable_ibex = False
#deprecated
iris_options.certify_region_with_sos_during_generation = False
iris_options.certify_region_with_sos_after_generation = False

seed_point = np.array([1.0, -1.2, -0.45])
plant.SetPositions(plant.GetMyMutableContextFromRoot(context), seed_point)
region = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context), iris_options)
print(f"{region.A()}t <= {region.b()}")
visualizer.plot_regions([region], ellipses=None, region_suffix='1', opacity=0.8)
print(region.PointInSet(seed_point))


In [45]:
if do_viz:
    visualizer.plot_regions(iris_generator.regions, region_suffix = '_python_snopt')

In [ ]:
# refine with python
do_linesearch_cert = True
if do_linesearch_cert:
    iris_generator.certify_and_adjust_regions_by_linesearch(1e-5)

Starting Region 1/5
min_eps = [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
cur_eps = [0.56916636 0.77870386 0.56916636 0.56916636 0.77870386 0.56916636
 0.22000147 0.25927221 0.17511844 0.11487268 0.10200084 0.10008813
 0.1746615  0.31397342 0.33108215]
max_eps = [1.13833271 1.55740772 1.13833271 1.13833271 1.55740772 1.13833271
 0.44000293 0.51854441 0.35023687 0.22974536 0.20400167 0.20017626
 0.349323   0.62794684 0.66216431]
Biggest difference: 1.5574077246549023
Smallest difference: 0.20017626488520598
Fixed eps is success

min_eps = [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
cur_eps = [0.28458318 0.38935193 0.28458318 0.28458318 0.38935193 0.28458318
 0.11000073 0.1296361  0.08755922 0.05743634 0.05100042 0.05004407
 0.08733075 0.15698671 0.16554108]
max_eps = [0.56916636 0.77870386 0.56916636 0.56916636 0.77870386 0.56916636
 0.22000147 0.25927221 0.17511844 0.11487268 0.10200084 0.10008813
 0.1746615  0.31397342 0.33108215]
Biggest difference: 0.7787038623274511
Smallest 

In [ ]:
if do_viz:
    visualizer.plot_regions(iris_generator.linesearch_regions, region_suffix = '_new_by_linesearch')

In [22]:
def create_L1_normball(dim, origin = None, scaling = 1.0):
    if origin is None:
        origin = np.zeros((dim,))
    def decimalToBinary(n, numberdigits):
        num_zeros = numberdigits
        for idx in range(numberdigits-1, -1, -1):
            if n >= 2**idx:
                num_zeros = numberdigits - idx -1
                break
        zeros = "0"*num_zeros if num_zeros<numberdigits else "0"*(numberdigits-1)

        return bin(n).replace("0b", zeros)

    #create norm ball"
    values = [-1.0, 1.0]
    normals = []
    dim = 3
    b = scaling/np.sqrt(dim)*np.ones((2**dim,))
    for idx in range(2**dim):
        template = decimalToBinary(idx, dim)
        normal = np.zeros((dim,))
        for digit, value in enumerate(template):
            normal[digit] = 1.0 if int(value) else -1
        normals.append(normal)
        A = np.array(normals)
    b += A@origin.reshape(-1,1).reshape(-1,)
    return HPolyhedron(A,b)


In [24]:
region = create_L1_normball(3, origin = seed_point, scaling= 1.0)
visualizer.plot_regions([region], ellipses=None, region_suffix='l1', opacity=0.8)

In [69]:
A.shape

(8, 3)

In [70]:
b.shape


(8,)